In [1]:
import numpy as np
import pandas as pd
import sys
import os
import fur.path_assistant as path_assistant
shift = path_assistant.PathAssistant('shift_03_16_2020',
    ignore_files=['RefCurve_2020-03-16_0_084020_test.Wfm.bin',
                  'RefCurve_2020-03-16_153_155200.Wfm.bin'])
from config import get_from_config, save_to_config
from scipy.interpolate import interp1d
from lattice.summary_in_undulator import get_summary_in_undulator
from lattice import lattice
lattice_file = shift.get_6dsim_dir().fi("IOTA_1NL_100MeV_v8.6.1.4.6ds_data.txt")

In [2]:
V = 360 # V rf also used below in zero-current section
sz_to_dpp = lattice.get_dpp(1,V)

In [3]:
from fur.jsonstorage import JsonStorage
js16 = JsonStorage(shift.get_results_dir().fi('jsonstorage_03_16_2020.json'))
dP_Psep = js16.get("dP_Psep")
momApFac = js16.get("momApFactor")

In [4]:
und_summary = get_summary_in_undulator(
    lattice_file,
    np.ones(8),
    np.ones(8),
    0,
    0
    )
Dx, betax, betay, alphax, alphay = und_summary[0].loc['Middle', ['Dispersion_cm_X', 'Beta_cm_X', 'Beta_cm_Y',
                          'Alpha_X', 'Alpha_Y']]
Dx, betax, betay, alphax, alphay

/home/ilobach/fur/env/lib/python3.6/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ilobach/fur/lattice/lattice.py:173: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(e_um*1e4*beta_cm+(1e4*dispersion_cm*dpp)**2)


(101.20460727716554,
 203.63015539385736,
 98.4141815632213,
 1.2502870273577453,
 -0.8742874700127816)

In [5]:
def newval(name, val, unit):
    return (r'\newcommand{\val'+name+r'}{\SI{'
              +val
              +r'}{'
              +unit
              +r'}}'+'\n')

In [6]:
cur_to_sum_channel = get_from_config("Beam_current_to_Sum_channel_ampl_V/mA")
sum_channel_to_photoelectrons =\
    get_from_config('sum_channel_to_photoelectrons')
meas_photons_per_electron =\
    get_from_config("Measured_photons_per_electron")
meas_photons_per_electron
cspeed = get_from_config("c_m/s")
Ibeam = get_from_config("NominalBeamCurrent_mA")
N_to_I = 1/sum_channel_to_photoelectrons/cur_to_sum_channel
N0 = Ibeam/N_to_I
Ku = get_from_config("K_peak")
gamma = get_from_config("gamma")
Ebeam = 0.511*gamma
Ku_error = get_from_config("Ku_error")
Ebeam_error= get_from_config("Ebeam_error_MeV")
und_per = get_from_config("Undulator_period_m")
lambda1 = und_per/2/gamma**2*(1+Ku**2/2)*1e6
zobs = get_from_config("z_obs_m")
inch = 0.0254 # m
apx_mrad = 1e3*inch/zobs
apy_mrad = apx_mrad/np.sqrt(2)
integrator_to_photoelectrons =\
    get_from_config("integrator_amp_to_photoelectrons")
hybrid_balance_error = get_from_config("HybridBalanceError")
hybrid_errorbar = 2*hybrid_balance_error
xi = get_from_config("integrator_amp_to_photoelectrons")\
    /get_from_config("sum_channel_to_photoelectrons")/2
test_light_source_theta = get_from_config("test_light_source_theta")
noise_level = get_from_config("noise_level")
noise_level_V2 = get_from_config("noise_level_V2")
varN_error_fit = get_from_config("varN_error_fit")

sh10 = path_assistant.PathAssistant('shift_03_10_2020')
from fur.jsonstorage import JsonStorage
js10 = JsonStorage(sh10.get_results_dir().fi('jsonstorage_03_10_2020.json'))

observed_M = js10.get("observed_M")
MerrND = js10.get("M_err")

# now from rcc cluster
ey_with_div = 0.008437 #js10.get("ey_with_div")
ey_without_div = 0.018277246293550275 #js10.get("ey_without_div")

ey_with_div_top = js10.get("ey_with_div_top") 
ey_with_div_bottom = js10.get("ey_with_div_bottom")
ey_without_div_top = js10.get("ey_without_div_top") 
ey_without_div_bottom = js10.get("ey_without_div_bottom")

# now from rcc cluster
ey_with_div_err = 0.0014710308562775852#(ey_with_div_top + ey_with_div_bottom)/2
ey_without_div_err = 0.0013249322830276734 # (ey_without_div_top + ey_without_div_bottom)/2

szeff_ND = js10.get("szeff_cm")
sz_ND = js10.get("sz_cm")
ex_ND = js10.get("ex")
dpp_ND = js10.get("dpp")
IbeamND = js10.get("Ibeam")
ey_lt_rec = js10.get("ey_lifetime_rec")
ey_lt_rec_err = js10.get("ey_lifetime_rec_err")
lifetimeND = js10.get("Lifetime")

In [7]:
shift_03_10_2020 = path_assistant.PathAssistant('shift_03_10_2020')
lattice_file_10 = shift_03_10_2020.get_6dsim_dir().fi("IOTA_1NL_100MeV_v8.6.1.3.6ds_data.txt")
lattice_df_10 = lattice.read_lattice_file(lattice_file_10)
emmitance_6D = {
        "ex_um": ex_ND,
        "ex_err_um": 0,
        "ey_um": 0,
        "ey_err_um": 0,
        "dp/p": dpp_ND,
        "dp/p_err": 0
    }
beamDivXND, dDx_ND  =\
lattice.get_undulator_df(lattice_df_10, emmitance_6D).loc['Middle',
                                                          ['Angle_spread_rad_X','dDx']]

In [8]:
bp_df = pd.read_csv(
    shift.get_results_dir().fi(f"beam_params_vs_current_round.csv"))
sx_func = interp1d(bp_df["N"], bp_df['Sigma_um_X'],
                   bounds_error=False, fill_value="extrapolate")
sy_func = interp1d(bp_df["N"], bp_df['Sigma_um_Y'],
                   bounds_error=False, fill_value="extrapolate")
sz_func = interp1d(bp_df["N"], bp_df['sz_um'],
                   bounds_error=False, fill_value="extrapolate")
ex_func = interp1d(bp_df["N"], bp_df['ex_um'],
                   bounds_error=False, fill_value="extrapolate")
ey_func = interp1d(bp_df["N"], bp_df['ey_um'],
                   bounds_error=False, fill_value="extrapolate")
dpp_func = interp1d(bp_df["N"], bp_df['dp/p'],
                    bounds_error=False, fill_value="extrapolate")
sz = sz_func(N0*IbeamND/Ibeam)
ex = ex_func(N0*IbeamND/Ibeam)
ey = ey_func(N0*IbeamND/Ibeam)
dpp = dpp_func(N0*IbeamND/Ibeam)
emmitance_6D = {
        "ex_um": ex,
        "ex_err_um": 0,
        "ey_um": ey,
        "ey_err_um": 0,
        "dp/p": dpp,
        "dp/p_err": 0
    }
lattice_df = lattice.read_lattice_file(lattice_file)
beamDivRoundX, beamDivRoundY = lattice.get_undulator_df(lattice_df, emmitance_6D)\
    .loc['Middle', ['Angle_spread_rad_X', 'Angle_spread_rad_X']].values
beamDivRoundX, beamDivRoundY

(0.00042759394782892053, 0.00042759394782892053)

In [9]:
res_df_loaded = \
    pd.read_csv(shift.get_results_dir().fi('meas_ROUND_03_16_2020.csv'),
                index_col=0)
res_df_ROUND = res_df_loaded[res_df_loaded['N']<2.25e7]
res_df_loaded = \
    pd.read_csv(shift.get_results_dir().fi('meas_FLAT_03_16_2020.csv'),
                index_col=0)
res_df_FLAT = res_df_loaded[res_df_loaded['N']<2.4e7]
MrMf = []
for res_df in [res_df_ROUND, res_df_FLAT]:
    res_df = res_df.sort_values(by='N')
    varN = np.interp(N0, res_df['N'], res_df['varN'])
    M = N0**2/(varN-N0)
    MrMf.append(M)
Mr, Mf = MrMf

## Re-calculate predicted photons per electron

In [10]:
# from wiggler_radiation.Wigrad.wigrad_generator import \
#     generate_wr_sim_with_wigrad_results
# wr_sim = generate_wr_sim_with_wigrad_results(
# K_peak_in=Ku,
# gamma_in=gamma)
# ampx3d = wr_sim.get_amplittude_3D(polarization='x')
# mesh = get_from_config("radiation_mesh")
# xmin, xmax, _ = mesh[0]
# xmin = xmin/zobs
# xmax = xmax/zobs
# ymin, ymax, _ = mesh[1]
# ymin = ymin/zobs
# ymax = ymax/zobs
# lmin, lmax, _ = mesh[2]
# dax = (xmax-xmin)/(mesh[0][2]-1)
# day = (ymax-ymin)/(mesh[1][2]-1)
# dl = (lmax-lmin)/(mesh[2][2]-1)
# tot =dax*day*dl*np.sum(np.absolute(ampx3d)**2)
# save_to_config("Predicted_photons_per_electron_wigrad", tot)

In [11]:
photonFluxMeas = get_from_config("Measured_photons_per_electron")
photonFluxTheor = get_from_config("Predicted_photons_per_electron_wigrad")

In [12]:
# Vertical size and emittance of flat beam
meas_FLAT = pd.read_csv(
    shift.get_results_dir().fi('meas_FLAT_03_16_2020.csv'), index_col=0)
meas_FLAT = meas_FLAT.sort_values('N')
sy_flat = np.interp(N0, meas_FLAT['N'], meas_FLAT['sy_rec'])
ey_flat = np.interp(N0, meas_FLAT['N'], meas_FLAT['ey_rec'])

In [13]:
Nu = 10.5

## zero-current 

In [14]:
iota_period_sec = get_from_config("IOTA_revolution_period")
c = 2.9979e10
gamma = get_from_config("gamma")
hbar_c = 0.1973269804e-4
rho = 70
Ec = 3/2*hbar_c*gamma**3/rho
Et = 88.9e-7*(gamma*0.511)**4/rho
Scale = 15*np.sqrt(3)/8*Et
alpha = get_from_config("ring_alpha")
E0 = gamma*511000
f = 30e6
h = 4
v0 = V/E0
S = Scale/E0
eta = alpha - 1/gamma**2
w = 2*np.pi*h*eta
ts = 0.836
JE = 2*E0*iota_period_sec/ts/Et
delta_rms = 0.62e-6*gamma/np.sqrt(JE*rho/100)
tau_rms = 1e9*delta_rms/(f*2*np.pi*np.sqrt(V/(2*np.pi*E0*h*np.abs(eta))))
sz_zero_cur = tau_rms*1e-9*c
delta_rms

8.349010114420189e-05

In [15]:
nrev = 7.5e6*1.5e-3

In [16]:
k0 = 2*np.pi/lambda1
slim = 1/(2*k0*np.sqrt(1+Ku**2/2)/(gamma*np.sqrt(Nu)))
exlim = 1000*slim**2/(1e4*betax)
eylim = 1000*slim**2/(1e4*betay)
slim

46.03699576114513

In [17]:
fwhm = get_from_config("fundamental_fwhm")

In [18]:
res = \
    r'% Do not edit this file. It is generated by a Python script.'+'\n'\
    +r'\ProvidesPackage{numericalvalues}'+'\n'\
    +r'\usepackage{siunitx}'+'\n'+'\n'+'\n'

res += newval("fwhm", f"{fwhm:.2f}", r"\micro m")
res += newval("slim", f"{10*((slim+5)//10):.0f}", r"\micro m")
res += newval("exlim", f"{exlim:.1f}", "nm")
res += newval("eylim", f"{eylim:.1f}", "nm")
res += newval("dppSep", f"{dP_Psep:.1e}", "")
res += newval("MomentumApertureFactor", f"{momApFac:.1f}", "")
res += newval("MomApEff", f"{dP_Psep*momApFac:.1e}", "")
res += newval("lifetimeND", f"{lifetimeND:.0f}", "s")
res += newval("eyLifetimeRecND", f"{ey_lt_rec:.1f}", "nm")
res += newval("eyLifetimeRecNDDimLess", f"{ey_lt_rec:.1f}", "")
res += newval("eyLifetimeRecNDErr", f"{ey_lt_rec_err:.1f}", "nm")
res += newval("Vrf", f"{V:.0f}", "V")
res += newval("szTospDimLess", f"{sz_to_dpp:.1e}", "")
res += newval("tr", "85", "ns")
res += newval("SLMroundErr", "8", "nm")
res += newval("SLMflatErr", "50", "nm")
res += newval("revolutionPeriodDimLess", f"{iota_period_sec*1e9:.1f}", "")
res += newval("revolutionPeriod", f"{iota_period_sec*1e9:.1f}", "ns")
res += newval("szeffND", f"{szeff_ND:.1f}", r"cm")
res += newval("szND", f"{sz_ND:.1f}", r"cm")
res += newval("dppND", f"{dpp_ND:.1e}", r"")
res += newval("exND", f"{ex_ND:.2f}", r"\micro m")
res += newval("IbeamND", f"{IbeamND:.2f}", "mA")
res += newval("beamDivRoundX", f"{1000*beamDivRoundX:.2f}", "mrad")
res += newval("beamDivRoundY", f"{1000*beamDivRoundY:.2f}", "mrad")
res += newval("beamDivXND", f"{1000*beamDivXND:.2f}", "mrad")
res += newval("dDxND", f"{dDx_ND:.2f}", "")
res += newval("observedM", f"{observed_M:.2e}", "")
res += newval("MerrND", f"{MerrND:.1e}", "")
res += newval("eyWithDivDimLess", f"{1000*ey_with_div:.1f}", r"")
res += newval("eyWithoutDivDimLess", f"{1000*ey_without_div:.1f}", r"")
res += newval("eyWithDivErr", f"{1000*ey_with_div_err:.1f}", r"nm")
res += newval("eyWithoutDivErr", f"{1000*ey_without_div_err:.1f}", r"nm")
res += newval("varNerror", f"{varN_error_fit:.1e}", r"")
res += newval("noiseLevel", f"{noise_level:.1e}", r"")
res += newval("noiseLevelV", f"{noise_level_V2:.1e}", r"V^2")
res += newval("theta", f"{test_light_source_theta:.2e}", r"")
res += newval("thetarms", f"{np.sqrt(test_light_source_theta):.2e}", r"")
res += newval("xi", f"{xi:.2f}", r"")
res += newval("Dx", f"{Dx:.0f}", r"cm")
res += newval("betax", f"{betax:.0f}", r"cm")
res += newval("betay", f"{betay:.0f}", r"cm")
res += newval("alphax", f"{alphax:.2f}",r'')
res += newval("alphay", f"{alphay:.2f}",r'')
res += newval("szZeroCur", f"{sz_zero_cur:.0f}", r"cm" )
res += newval("spZeroCur", f"{delta_rms:.1e}", r'')
res += newval("wfLen", "1.5", "ms")
res += newval("Nrev", f"{nrev:.0f}", "")
res += newval("EbeamDimLess", f"{Ebeam:.1f}", "")
res += newval("Ebeam", f"{Ebeam:.1f}", 'MeV')
res += newval("Ku", f"{Ku:.1f}", '')
res += newval("lambdaone", f"{lambda1:.2f}", r'\micro m')
res += newval("Ibeam", f"{Ibeam:.1f}", 'mA')
res += newval("Mround", f"{Mr:.1e}", '')
res += newval("Mflat", f"{Mf:.1e}", '')
res += newval("zobs", f"{zobs:.1f}", 'm')
res += newval("semiApertureX", f"{apx_mrad:.1f}", 'mrad')
res += newval("semiApertureY", f"{apy_mrad:.1f}", 'mrad')
res += newval("oneOverGammaNu", f"{np.sqrt(1+Ku**2/2)/gamma/np.sqrt(Nu)/2*1e3:.1f}", 'mrad')
res += newval("photonFluxMeasDimLess", f"{photonFluxMeas:.1e}", '')
res += newval("photonFluxTheorDimLess", f"{photonFluxTheor:.1e}", '')
res += newval("photonFluxMeas", f"{photonFluxMeas:.1e}", 'photoelectrons/electron')
res += newval("photonFluxTheor", f"{photonFluxTheor:.1e}", 'photoelectrons/electron')
res += newval("AmpToPhotoel", f"{integrator_to_photoelectrons:.2e}",
              "photoelectrons/V")
res += newval("Kuerror", f"{Ku_error:.2f}", '')
res += newval("Ebeamerror", f"{Ebeam_error:.0f}", "MeV")
res += newval("HybridErrorbar", f"{hybrid_errorbar*100:.0f}",r'\percent')
res += newval("NundPer", f"{Nu:.1f}", "")


for kind in ["round", "flat"]:
    res+='\n'+r'% '+kind+" beam parameters"+'\n'
    bp_df = pd.read_csv(
        shift.get_results_dir().fi(f"beam_params_vs_current_{kind}.csv"))
    sx_func = interp1d(bp_df["N"], bp_df['Sigma_um_X'],
                       bounds_error=False, fill_value="extrapolate")
    sy_func = interp1d(bp_df["N"], bp_df['Sigma_um_Y'],
                       bounds_error=False, fill_value="extrapolate")
    sz_func = interp1d(bp_df["N"], bp_df['sz_um'],
                       bounds_error=False, fill_value="extrapolate")
    ex_func = interp1d(bp_df["N"], bp_df['ex_um'],
                       bounds_error=False, fill_value="extrapolate")
    ey_func = interp1d(bp_df["N"], bp_df['ey_um'],
                       bounds_error=False, fill_value="extrapolate")
    dpp_func = interp1d(bp_df["N"], bp_df['dp/p'],
                        bounds_error=False, fill_value="extrapolate")
    cspeed = get_from_config("c_m/s")
    Ibeam = get_from_config("NominalBeamCurrent_mA")
    N_to_I = 1/sum_channel_to_photoelectrons/cur_to_sum_channel
    sx = sx_func(N0)
    sy = sy_func(N0) if (kind == "round") else sy_flat
    sz = sz_func(N0)
    ex = ex_func(N0)
    ey = ey_func(N0) if (kind == "round") else ey_flat
    dpp = dpp_func(N0)
    for sym, var in zip(["sx", "sy"],[sx, sy]):
        res+= newval(sym+kind, f"{var:.1f}", r'\micro m')
    for sym, var in zip(["ex", "ey"],[ex*1e3, ey*1e3]):
        res+=newval(sym+kind, f"{var:.1f}", r'nm')
    res+=newval('sz'+kind, f"{sz/10000:.0f}", r'cm')
    res+=newval('st'+kind, f"{sz/1000000/cspeed*1e9:.2f}", r'ns')
    res+=newval('dpp'+kind, f"{dpp:.1e}", '')
print(res)

% Do not edit this file. It is generated by a Python script.
\ProvidesPackage{numericalvalues}
\usepackage{siunitx}


\newcommand{\valfwhm}{\SI{0.14}{\micro m}}
\newcommand{\valslim}{\SI{50}{\micro m}}
\newcommand{\valexlim}{\SI{1.0}{nm}}
\newcommand{\valeylim}{\SI{2.2}{nm}}
\newcommand{\valdppSep}{\SI{2.8e-03}{}}
\newcommand{\valMomentumApertureFactor}{\SI{0.7}{}}
\newcommand{\valMomApEff}{\SI{2.0e-03}{}}
\newcommand{\vallifetimeND}{\SI{559}{s}}
\newcommand{\valeyLifetimeRecND}{\SI{9.6}{nm}}
\newcommand{\valeyLifetimeRecNDDimLess}{\SI{9.6}{}}
\newcommand{\valeyLifetimeRecNDErr}{\SI{1.2}{nm}}
\newcommand{\valVrf}{\SI{360}{V}}
\newcommand{\valszTospDimLess}{\SI{9.1e-06}{}}
\newcommand{\valtr}{\SI{85}{ns}}
\newcommand{\valSLMroundErr}{\SI{8}{nm}}
\newcommand{\valSLMflatErr}{\SI{50}{nm}}
\newcommand{\valrevolutionPeriodDimLess}{\SI{133.3}{}}
\newcommand{\valrevolutionPeriod}{\SI{133.3}{ns}}
\newcommand{\valszeffND}{\SI{29.5}{cm}}
\newcommand{\valszND}{\SI{31.9}{cm}}
\newcommand{\valdppND}

In [19]:
with open("/home/ilobach/FUR_Run2_experiment_paper/numericalvalues.sty", 'w') as f:
    f.write(res)